In [1]:
import pandas as pd

In [3]:
faults = pd.read_csv("../data/J1939Faults.csv")
diagnostics = pd.read_csv("../data/VehicleDiagnosticOnboardData.csv", nrows = 100)

C:\Users\jdavis\AppData\Local\Temp\ipykernel_31300\1152306463.py:1: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  faults = pd.read_csv("../data/J1939Faults.csv")


In [5]:
faults.shape

(1187335, 20)

In [7]:
faults_diagnostics = pd.merge(faults, diagnostics, 
                              left_on='RecordID', right_on='FaultId', 
                              how='left')

In [9]:
drop_list = ['ESS_Id', 
             'actionDescription', 
             'ecuSoftwareVersion', 
             'ecuSerialNumber', 
             'ecuModel', 
             'ecuMake', 
             'ecuSource', 
             'faultValue',
             'Latitude',
             'LocationTimeStamp',
             'Longitude',
             'Id',
             'MCTNumber']

faults_diagnostics = faults_diagnostics.drop(columns=drop_list)

In [13]:
var_filter = 1395
filter = faults_diagnostics[faults_diagnostics['EquipmentID'] == var_filter]
#var_filter = ['False', 'True']
#filter = faults_diagnostics[~faults_diagnostics['Value'].isin(var_filter)]
filter.head(30)

,RecordID,EventTimeStamp,eventDescription,spn,fmi,active,activeTransitionCount,EquipmentID,Name,Value,FaultId
33401,34428,2015-04-26 01:42:41.000,Low (Severity Low) Engine Coolant Level,111,17,True,1,1395,NaN,NaN,NaN
33402,34429,2015-04-26 01:46:16.000,Low (Severity Low) Engine Coolant Level,111,17,False,1,1395,NaN,NaN,NaN
60978,62963,2015-05-22 13:03:07.000,Abnormal Rate of Change Aftertreatment 1 Intak...,3216,10,True,1,1395,NaN,NaN,NaN
62220,64205,2015-05-23 19:40:38.000,Abnormal Rate of Change Aftertreatment 1 Intak...,3216,10,False,1,1395,NaN,NaN,NaN
75233,77534,2015-06-04 21:08:40.000,Abnormal Rate of Change Aftertreatment 1 Intak...,3216,10,True,1,1395,NaN,NaN,NaN
79863,82164,2015-06-09 15:55:21.000,Abnormal Rate of Change Aftertreatment 1 Intak...,3216,10,True,1,1395,NaN,NaN,NaN
80101,82402,2015-06-09 21:12:29.000,Abnormal Update Rate Aftertreatment 1 Intake NOx,3216,9,True,1,1395,NaN,NaN,NaN
80108,82409,2015-06-09 21:47:22.000,Abnormal Update Rate Aftertreatment 1 Intake NOx,3216,9,False,1,1395,NaN,NaN,NaN
80109,82410,2015-06-09 21:47:22.000,Abnormal Rate of Change Aftertreatment 1 Intak...,3216,10,False,1,1395,NaN,NaN,NaN
91755,94056,2015-06-20 14:09:07.000,Incorrect Data Steering Wheel Angle,1807,2,True,127,1395,NaN,NaN,NaN


In [ ]:
faults_diagnostics.shape

In [ ]:
faults_diagnostics['spn'].value_counts()

In [15]:
filter.to_csv('equipmentID_1395.csv', index=False)

In [ ]:
faults.loc[0]

Looking at the first record, here is a breakdown of the important values.

* ESS_Id, actionDescription, ecuSoftwareVersion, ecuSerialNumber, ecuModel, ecuMake, ecuSource, faultValue, and MCTNumber are unlikely to provide any predictive value.
* We can see the time of the event in the **EventTimeStamp** column. Note that this may be different from the **LocationTimeStamp** value, which indicates when the Latitude/Longitude values were recorded.
* The **spn** and **fmi** columns together indicate the type of fault, and there may be a description of that fault in the **eventDescription** column, although this column is sometimes missing.
* Faults are recorded when the light goes on and when it goes off, which is indicated by the **active** column, with True indicating the light turning on and False indicating turning off. The number of times the code has been set or unset is in the **faultValue** column, although this value can be unreliable. 
* Each truck has an identifier, the **EquipmentID** value.
* Each record can be linked to the on-board diagnostics data through the **RecordID** column.

In [ ]:
diagnostics.head()

To get the on-board diagnostics at the time of the fault code, we can match the **RecordID** to the **FaultId**.

In [ ]:
diagnostics.loc[diagnostics['FaultId'] == 1]

This data is in long-format, so each FaultId can have potentially many diagnostic values.

**Note:** Not all diagnostic values are recorded for all faults, so you will have a large number of missing values.

For example, for the second fault code in our dataset, only the ignition status and lamp status were recorded.

In [ ]:
diagnostics.loc[diagnostics['FaultId'] == 2]

Finally, we can get a little bit more information about the different fault codes from the Service Fault Codes spreadsheet.

In [ ]:
sfc = pd.read_excel("../data/Service Fault Codes_1_0_0_167.xlsx")
sfc.head()

For a large number of fault codes, there are multiple records. For example, if we look at the rows for the first fault in our dataset, we see that there are two rows.

In [ ]:
(
    sfc
    .loc[sfc['SPN'] == 111]
    .loc[sfc['J1939 FMI'] == 17]
)

Or even more.

In [ ]:
(
    sfc
    .loc[sfc['SPN'] == 629]
    .loc[sfc['J1939 FMI'] == 12]
)